# Exercise - Data Exchange Formats with Python

In the following three exercises, you are asked to write Python code for reading
data from XML, JSON and RDF files and for querying the data using the XPath
and SPARQL query languages. Each subsection is dedicated to one of the three
data exchange formats. The tasks are rather basic and the goal is to refresh
your knowledge in Python in general and in particular in parsing those formats.

## 1 XML

This subsection is dedicated to the XML format. In particular, you are asked
to perform XPath queries on the Mondial dataset. This dataset includes world
geographic information integrated from the CIA World Factbook, the International
Atlas and the TERRA database, to name just the pre-dominant sources.

Please inspect the documents in '/input' manually (using a text editor) in order to explore
the structure.

You can also have a look at the [w3school XPath tutorial 2](https://www.w3schools.com/xml/xpath_intro.asp) to solve the following tasks.

In [14]:
"""
XML Tree Visualization Examples
Demonstrates various ways to visualize XML as a tree structure in Python
"""

import xml.etree.ElementTree as ET
from typing import Optional, List, Dict, Any
import pandas as pd
from pathlib import Path


def print_ascii_tree(element: ET.Element, prefix: str = "", is_last: bool = True):
    """Print XML tree as ASCII art"""
    # Print current element
    connector = "└── " if is_last else "├── "
    print(f"{prefix}{connector}{element.tag}")

    # Prepare prefix for children
    child_prefix = prefix + ("    " if is_last else "│   ")

    # Get children
    children = list(element)

    # Print attributes if any
    if element.attrib:
        for i, (key, value) in enumerate(element.attrib.items()):
            attr_connector = "└── " if i == len(
                element.attrib) - 1 and not children else "├── "
            print(f"{child_prefix}{attr_connector}@{key}: {value}")

    # Print text content if any
    if element.text and element.text.strip():
        text_connector = "└── " if not children else "├── "
        print(f"{child_prefix}{text_connector}text: {repr(element.text.strip())}")

    # Print children recursively
    for i, child in enumerate(children):
        is_last_child = (i == len(children) - 1)
        print_ascii_tree(child, child_prefix, is_last_child)


def get_tree_stats(element: ET.Element) -> Dict[str, Any]:
    """Get statistics about the XML tree"""
    def traverse(elem: ET.Element, depth: int = 0) -> Dict[str, Any]:
        stats = {
            'max_depth': depth,
            'total_elements': 1,
            'element_types': {elem.tag: 1},
            'attributes_count': len(elem.attrib),
            'text_nodes': 1 if elem.text and elem.text.strip() else 0
        }

        for child in elem:
            child_stats = traverse(child, depth + 1)
            stats['max_depth'] = max(
                stats['max_depth'], child_stats['max_depth'])
            stats['total_elements'] += child_stats['total_elements']
            stats['attributes_count'] += child_stats['attributes_count']
            stats['text_nodes'] += child_stats['text_nodes']

            # Merge element type counts
            for tag, count in child_stats['element_types'].items():
                stats['element_types'][tag] = stats['element_types'].get(
                    tag, 0) + count

        return stats

    return traverse(element)


def visualize_with_networkx(element: ET.Element, filename: str = "xml_tree.png"):
    """Create a graphical tree visualization using NetworkX and matplotlib"""
    try:
        import networkx as nx
        import matplotlib.pyplot as plt
    except ImportError:
        print("NetworkX and matplotlib required for graphical visualization")
        print("Install with: pip install networkx matplotlib")
        return

    def build_graph(elem: ET.Element, graph, parent_id=None, node_id=0):
        current_id = node_id
        label = elem.tag

        # Add attributes to label if present
        if elem.attrib:
            attrs = ", ".join(f"{k}={v}" for k, v in elem.attrib.items())
            label += f"\n({attrs})"

        # Add text content if present
        if elem.text and elem.text.strip():
            label += f"\n\"{elem.text.strip()}\""

        graph.add_node(current_id, label=label)

        if parent_id is not None:
            graph.add_edge(parent_id, current_id)

        next_id = current_id + 1
        for child in elem:
            next_id = build_graph(child, graph, current_id, next_id)

        return next_id

    # Create graph
    G = nx.DiGraph()
    build_graph(element, G)

    # Draw the tree
    plt.figure(figsize=(12, 8))
    pos = nx.spring_layout(G, k=2, iterations=50)

    # Draw nodes
    nx.draw(G, pos, with_labels=True, labels=nx.get_node_attributes(G, 'label'),
            node_color='lightblue', node_size=3000, font_size=8,
            font_weight='bold', edge_color='gray', arrows=True)

    plt.title("XML Tree Structure")
    plt.axis('off')
    plt.tight_layout()
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    plt.show()

    print(f"Tree visualization saved as {filename}")


def visualize_with_graphviz(element: ET.Element, filename: str = "xml_tree"):
    """Create a tree visualization using GraphViz"""
    try:
        from graphviz import Digraph
    except ImportError:
        print("GraphViz required for visualization")
        print("Install with: pip install graphviz")
        print("Also install GraphViz system package: https://graphviz.org/download/")
        return

    def build_dot_graph(elem: ET.Element, dot, parent_id=None, node_id=0):
        current_id = f"node_{node_id}"
        label = elem.tag

        # Add attributes to label if present
        if elem.attrib:
            attrs = "\\n".join(f"{k}={v}" for k, v in elem.attrib.items())
            label += f"\\n{attrs}"

        # Add text content if present
        if elem.text and elem.text.strip():
            label += f"\\n{elem.text.strip()}"

        dot.node(current_id, label=label)

        if parent_id is not None:
            dot.edge(parent_id, current_id)

        next_id = node_id + 1
        for child in elem:
            next_id = build_dot_graph(child, dot, current_id, next_id)

        return next_id

    # Create graph
    dot = Digraph(comment='XML Tree Structure')
    dot.attr(rankdir='TB')  # Top to bottom layout

    build_dot_graph(element, dot)

    # Save and render
    dot.render(filename, view=True, format='png')
    print(f"Tree visualization saved as {filename}.png")


def create_tree_dataframe(element: ET.Element) -> pd.DataFrame:
    """Convert XML tree to a pandas DataFrame for analysis"""
    def traverse_tree(elem: ET.Element, path: str = "", depth: int = 0) -> List[Dict]:
        rows = []

        # Current element
        row = {
            'path': path + elem.tag,
            'tag': elem.tag,
            'depth': depth,
            'has_children': len(list(elem)) > 0,
            'attributes': str(elem.attrib) if elem.attrib else None,
            'text_content': elem.text.strip() if elem.text and elem.text.strip() else None,
            'parent': path.rstrip('/') if path else None
        }
        rows.append(row)

        # Children
        for child in elem:
            child_path = f"{path}{elem.tag}/"
            rows.extend(traverse_tree(child, child_path, depth + 1))

        return rows

    rows = traverse_tree(element)
    return pd.DataFrame(rows)



"""Demonstrate XML tree visualization with the Mondial dataset"""
    # Create a sample XML for demonstration
sample_xml = """<?xml version="1.0" encoding="UTF-8"?>
<bookstore>
<book category="fiction">
    <title>Harry Potter</title>
    <author>J.K. Rowling</author>
</book>
<book category="non-fiction">
    <title>XML Guide</title>
    <author>John Doe</author>
</book>
</bookstore>"""
root = ET.fromstring(sample_xml)
print("Using sample XML for demonstration:")
print(sample_xml)

print("\n" + "="*60)
print("1. ASCII TREE VISUALIZATION")
print("="*60)
print_ascii_tree(root)

print("\n" + "="*60)
print("2. TREE STATISTICS")
print("="*60)
stats = get_tree_stats(root)
for key, value in stats.items():
    print(f"{key}: {value}")

print("\n" + "="*60)
print("3. TREE AS DATAFRAME")
print("="*60)
df = create_tree_dataframe(root)
print(df.head(10))


Using sample XML for demonstration:
<?xml version="1.0" encoding="UTF-8"?>
<bookstore>
<book category="fiction">
    <title>Harry Potter</title>
    <author>J.K. Rowling</author>
</book>
<book category="non-fiction">
    <title>XML Guide</title>
    <author>John Doe</author>
</book>
</bookstore>

1. ASCII TREE VISUALIZATION
└── bookstore
    ├── book
    │   ├── @category: fiction
    │   ├── title
    │   │   └── text: 'Harry Potter'
    │   └── author
    │       └── text: 'J.K. Rowling'
    └── book
        ├── @category: non-fiction
        ├── title
        │   └── text: 'XML Guide'
        └── author
            └── text: 'John Doe'

2. TREE STATISTICS
max_depth: 2
total_elements: 7
element_types: {'bookstore': 1, 'book': 2, 'title': 2, 'author': 2}
attributes_count: 2
text_nodes: 4

3. TREE AS DATAFRAME
                    path        tag  depth  has_children  \
0              bookstore  bookstore      0          True   
1         bookstore/book       book      1          True  

### 1.1 Load the dataset and inspect the schema.

We use the [pandas](https://pandas.pydata.org/) library to load and process XML files in Python.

Pandas offers the function [read_xml](https://pandas.pydata.org/docs/reference/api/pandas.read_xml.html) to read XML documents into a pandas DataFrame object.

In this first task, load the dataset from 'input/mondial-3.0.xml' and print the names of the nodes below the root node.

In [3]:
import pandas as pd

# Load the file and return the columns. The columns of the dataframe represent nodes of the input XML.
df_nodes = pd.read_xml("../Task/input/mondial-3.0.xml", xpath = "/*")
df_nodes.columns

Index(['continent', 'country', 'organization', 'mountain', 'desert', 'island',
       'river', 'sea', 'lake'],
      dtype='object')

### 1.1a Basic Element Selection

Let's start with the basics of XPath. First, select all continent elements from the XML document and print how many continents there are.

Hint: The `//` selector finds all elements of a specific type anywhere in the document, regardless of their position in the hierarchy.

In [4]:
# Select all continent elements and count them
df_continents = pd.read_xml("../Task/input/mondial-3.0.xml", xpath="//continent")
print(f"Number of continents: {len(df_continents)}")
df_continents.head()

Number of continents: 5


,id,name
0,f0_119,Europe
1,f0_123,Asia
2,f0_126,America
3,f0_129,Australia/Oceania
4,f0_132,Africa


### 1.1b Attribute Selection

Now, let's learn how to select attributes. Get the names of all continents using attribute selection.

Hint: The `@` symbol is used to select attributes in XPath. You can access attributes directly or select elements and then access their attributes using pandas column syntax.

In [5]:
from lxml import etree
import pandas as pd

tree = etree.parse('../Task/input/mondial-3.0.xml')

# Get the names of all continents
print(tree.xpath('//continent/@name'))

['Europe', 'Asia', 'America', 'Australia/Oceania', 'Africa']


### 1.1c Filtering by Attribute Value

Let's practice filtering elements by their attribute values. Find the continent element that has the name 'Europe'.

Hint: Square brackets `[]` are used for filtering in XPath. Inside the brackets, you can specify conditions using attributes and comparison operators.

In [6]:
# Find the continent with name 'Europe'
df_europe_continent = pd.read_xml("../Task/input/mondial-3.0.xml", xpath="//continent[@name='Europe']")
print("Europe continent details:")
df_europe_continent

Europe continent details:


,id,name
0,f0_119,Europe


### 1.1d Positional Selection

XPath allows you to select elements by their position. Get the name of the first country in the dataset.

Hint: Position indexing in XPath uses square brackets with numbers. XPath positions start counting from 1, not 0.

In [7]:
# Get the name of the first country
df_first_country = pd.read_xml("../Task/input/mondial-3.0.xml", xpath="//country[1]")
print(f"First country name: {df_first_country['name'].iloc[0]}")
df_first_country[['name', 'population', 'capital']]

First country name: 
       Albania
     


,name,population,capital
0,\n Albania\n,3249136,f0_1461


### 1.1e Countries in Europe (Complex XPath)

Select all `country` elements encompassed by the continent named 'Europe' and print their names.


In [8]:
# Countries that belong to Europe
df_europe_preview = pd.read_xml("../Task/input/mondial-3.0.xml",
                                xpath="/mondial/country[encompassed/@continent=/mondial/continent[@name='Europe']/@id]")
df_europe_preview['name']


0                    \n       Albania\n     
1                    \n       Andorra\n     
2                    \n       Austria\n     
3                    \n       Belarus\n     
4                    \n       Belgium\n     
5     \n       Bosnia and Herzegovina\n     
6                   \n       Bulgaria\n     
7                    \n       Croatia\n     
8             \n       Czech Republic\n     
9                    \n       Denmark\n     
10                   \n       Estonia\n     
11             \n       Faroe Islands\n     
12                   \n       Finland\n     
13                    \n       France\n     
14                   \n       Germany\n     
15                 \n       Gibraltar\n     
16                    \n       Greece\n     
17                  \n       Guernsey\n     
18                  \n       Holy See\n     
19                   \n       Hungary\n     
20                   \n       Iceland\n     
21                   \n       Ireland\n     
22        

### 1.1f Conditional Selection (and/or)

Select all countries that (are in Europe and have a population greater than 20,000,000) or have a total area greater than 1,000,000, and print their names.


In [9]:
# Countries in Europe with population > 20,000,000 OR total_area > 1,000,000
df_conditional = pd.read_xml("../Task/input/mondial-3.0.xml",
                             xpath="/mondial/country[(encompassed/@continent=/mondial/continent[@name='Europe']/@id and @population > 20000000) or @total_area > 1000000]")
df_conditional['name']


0             \n       France\n     
1            \n       Germany\n     
2              \n       Italy\n     
3             \n       Poland\n     
4            \n       Romania\n     
5             \n       Russia\n     
6              \n       Spain\n     
7            \n       Ukraine\n     
8     \n       United Kingdom\n     
9              \n       China\n     
10             \n       India\n     
11         \n       Indonesia\n     
12              \n       Iran\n     
13         \n       Kazakstan\n     
14          \n       Mongolia\n     
15      \n       Saudi Arabia\n     
16            \n       Turkey\n     
17         \n       Argentina\n     
18           \n       Bolivia\n     
19            \n       Brazil\n     
20            \n       Canada\n     
21          \n       Colombia\n     
22            \n       Mexico\n     
23              \n       Peru\n     
24     \n       United States\n     
25         \n       Australia\n     
26         \n       Greenland\n     
2

Now that you've practiced the basics of XPath, let's move on to more complex queries that combine multiple concepts:

## 2 JSON

### 2.1 Load the dataset

We use the [pandas](https://pandas.pydata.org/) library to load and process JSON files in Python.

Pandas offers the function [read_json](https://pandas.pydata.org/docs/reference/api/pandas.read_json.html) to read JSON documents into a pandas DataFrame object.

Load the dataset from 'input/mondial-3.0-europe-countries.json' and calculate the total number of inhabitants of all countries in the file.

Hint:
- Double-check the format of the JSON document before loading the file.

In [11]:
# Load and inspect the dataset.
df_countries = pd.read_json('../Task/input/gpu_products.json')
df_countries.head()

,id,name,brand,model,chipset,memory_gb,memory_type,clock_speed_mhz,tdp_w,launch_date,price_usd,description
0,gpu-001,NVIDIA GeForce RTX 4070 Ti,NVIDIA,RTX 4070 Ti,AD104,12,GDDR6X,2310,285,2023-01-05,799,"GeForce RTX 4070 Ti with 12GB GDDR6X, boost up..."
1,gpu-002,AMD Radeon RX 7800 XT,AMD,RX 7800 XT,Navi 32,16,GDDR6,2124,263,2023-09-06,499,"Radeon RX 7800 XT with 16 GB GDDR6, boost ~2.4..."
2,gpu-003,NVIDIA GeForce RTX 4090,NVIDIA,RTX 4090,AD102,24,GDDR6X,2235,450,2022-10-12,1599,Flagship RTX 4090 (24GB GDDR6X). Boost ~2.5 GH...
3,gpu-004,AMD Radeon RX 7600,AMD,RX 7600,Navi 33,8,GDDR6,2250,165,2023-05-25,269,RX 7600 with 8 GB GDDR6. Boost ~2.6 GHz. TBP ~...
4,gpu-005,NVIDIA GeForce RTX 4060,NVIDIA,RTX 4060,AD107,8,GDDR6,2460,115,2023-06-29,299,RTX 4060 8GB. Boost around 2.5 GHz. 115W TGP. ...


In [13]:
# Calculate the total population.
df_countries['price_usd'].sum()

7060